In [ ]:
import csv
from csv import DictReader
from typing import List, Optional
import re
import os

try:
    from colorama import Fore, Style, init
except ImportError:
    print("The colorama Library is not Installed. Install it Now..")
    os.system("pip install colorama")
    print("-------------------------------------------------------")
    print("installation completed")
    print("-------------------------------------------------------")
    print("")

CSV_HEADERS = ['ID', 'name', 'position', 'salary', 'email']

class Employee:
    def __init__(self, ID: int, name: str, position: str, salary: float, email: str):
        self.ID = ID
        self.name = name
        self.position = position
        self.salary = salary
        self.email = email

    def __str__(self):
        return f"ID: {self.ID}, Name: {self.name}, Position: {self.position}, Salary: {self.salary}, Email: {self.email}"

class EmployeeManager:
    def __init__(self, filename: str):
        self.filename = filename
        self.employees: List[Employee] = []
        self.load_or_create_csv()

    def load_or_create_csv(self):
        try:
            with open(self.filename, "r", encoding="utf8") as file_handle:
                csv_reader = DictReader(file_handle)
                for row in csv_reader:
                    employee = Employee(
                        ID=int(row['ID']),
                        name=row['name'],
                        position=row['position'],
                        salary=float(row['salary']),
                        email=row['email']
                    )
                    self.employees.append(employee)
            print(f"Loaded Data from {self.filename}.")
        except FileNotFoundError:
            print(f"{self.filename} NoT Found. Creating a new CSV file.")
            self.create_new_csv()
        except ValueError as e:
            print(f"Error: {e}.check CSV file.")
        except Exception as e:
            print(f"Error while loading: {e}")

    def create_new_csv(self):
        headers = CSV_HEADERS
        data = [headers]

        while True:
            row = input("Enter a row of data or 'exit' to finish: ")
            if row.lower() == 'exit':
                break
            data.append(row.split(','))

        with open(self.filename, mode='w', encoding="utf8") as file_handle:
            writer = csv.writer(file_handle)
            writer.writerows(data)
        print(f"{self.filename} created successfully.")

    def save_data(self):
        try:
            with open(self.filename, 'w', encoding="utf8") as file_handle:
                writer = csv.writer(file_handle)
                writer.writerow(CSV_HEADERS)
                for employee in self.employees:
                    writer.writerow([employee.ID, employee.name, employee.position, employee.salary, employee.email])
        except Exception as e:
            print(f"Error while saving data: {e}")

    def add_emp(self, ID: int, name: str, position: str, salary: float, email: str):
        if any(emp.ID == ID for emp in self.employees):
            print(f"Employee with ID {ID} already exists.")
            return
        new_employee = Employee(ID, name, position, salary, email)
        self.employees.append(new_employee)
        self.save_data()

    def update_data(self, ID: int, name: Optional[str] = None, position: Optional[str] = None,
                    salary: Optional[float] = None, email: Optional[str] = None):
        for employee in self.employees:
            if employee.ID == ID:
                if name is not None:
                    employee.name = name
                if position is not None:
                    employee.position = position
                if salary is not None:
                    employee.salary = salary
                if email is not None:
                    employee.email = email
                self.save_data()
                return
        print(f"Employee ID {ID} is NoT Found.")

    def del_data(self, ID: int):
        if not any(emp.ID == ID for emp in self.employees):
            print(f"ID {ID} is NoT Found Cannot delete.")
            return
        self.employees = [employee for employee in self.employees if employee.ID != ID]
        self.save_data()

    def search_data(self, ID: int) -> Optional[Employee]:
        for employee in self.employees:
            if employee.ID == ID:
                return employee
        return None

    def list_all_emp(self):
        if not self.employees:
            print("NoT Found.")
        else:
            for employee in self.employees:
                print(employee)

    def is_valid_email(self, email: str) -> bool:
        return re.match(r"[^@]+@[^@]+\.[^@]+", email) is not None

if __name__ == "__main__":
    filename = "employees.csv"  
    if os.path.exists(filename):
        print(f"{filename} exists. Loading existing data.")
        manager = EmployeeManager(filename)
    else:
        print(f"{filename} does NOT exist.")
        create_new = input("Would you like to create a new file?(yes/no): ").strip().lower()
        if create_new == 'yes':
            manager = EmployeeManager(filename)  
        else:
            print("EXIT.")
            exit()

    while True:
        print("\n\033[1;34mEMPLOYEE MANAGEMENT SYSTEM\033[0m")  
        print("\033[1;32m1. Add Employee\033[0m")              
        print("\033[1;32m2. Update Employee\033[0m")           
        print("\033[1;32m3. Delete Employee\033[0m")           
        print("\033[1;32m4. Search Employee\033[0m")           
        print("\033[1;32m5. List All Employees\033[0m")        
        print("\033[1;31m6. Exit\033[0m")

        choice = input("Choose an option: ")

        if choice == '1':  
            while True:
                ID_input = input("Enter ID: ")
                try:
                    ID = int(ID_input)  
                    break
                except ValueError:
                    print("Invalid input. Please enter ID again.")

            while True:
                name = input("Enter Name: ")
                if name.strip() and not name.isdigit(): 
                    break
                print("Invalid input. Please enter NAME again")

            while True:
                position = input("Enter Position: ")
                if position.strip() and not position.isdigit():  
                    break
                print("Invalid input. Please enter POSITION again.")

            while True:
                salary_input = input("Enter Salary: ")
                try:
                    salary = float(salary_input)  
                    break
                except ValueError:
                    print("Invalid input. Please enter SALARY again.")

            while True:
                email = input("Enter Email: ")
                if manager.is_valid_email(email):
                    break
                print("Invalid email format. Please enter an EMAIL again.")

            manager.add_emp(ID, name, position, salary, email)
            print("Employee Added Successfully.")

        elif choice == '2':
            while True:
                ID_input = input("Enter Employee ID to update: ")
                try:
                    ID = int(ID_input)  
                    break
                except ValueError:
                    print("Invalid input. Please enter ID again.")

            employee = manager.search_data(ID)
            if employee:
                print(f"Current details: {employee}")
                
                while True:
                    name = input("Enter new Name: ")
                    if name.strip() == "" or (not name.isdigit() and name.strip()):  
                        break
                    print("Invalid input. Please enter NAME again.")

                while True:
                    position = input("Enter new Position: ")
                    if position.strip() == "" or (not position.isdigit() and position.strip()):  
                        break
                    print("Invalid input. Please enter POSITION again.")
                
                salary_input = input("Enter new Salary: ")
                salary = None
                if salary_input:
                    try:
                        salary = float(salary_input)  
                    except ValueError:
                        print("Invalid input. Salary must be a number. TRY AGAIN")
                        continue

                email = input("Enter new Email: ")
                if email and not manager.is_valid_email(email):
                    print("Invalid email format. TRY AGAIN")
                    continue

                manager.update_data(ID, name or None, position or None, salary, email or None)
                print("Employee Updated Successfully.")
            else:
                print(f"Employee ID {ID} Not Found.")

        elif choice == '3':
            while True:
                ID_input = input("Enter Employee ID to delete: ")
                try:
                    ID = int(ID_input)  
                    break
                except ValueError:
                    print("Invalid input. Please enter ID again.")

            manager.del_data(ID)
            print("Employee Deleted Successfully.")

        elif choice == '4':
            while True:
                ID_input = input("Enter Employee ID to search: ")
                try:
                    ID = int(ID_input) 
                    break
                except ValueError:
                    print("Invalid input. Please enter ID again.")

            employee = manager.search_data(ID)
            
            if employee:
                print(f"Employee Found: {employee}")
            else:
                print(f"Employee ID {ID} NoT Found.")

        elif choice == '5':
            print("Listing All Employees:")
            manager.list_all_emp()

        elif choice == '6':
            print("Exiting the Employee Management System.")
            break

        else:
            print("Invalid choice. Please TRY AGAIN.")

employees.csv exists. Loading existing data.
Loaded Data from employees.csv.

EMPLOYEE MANAGEMENT SYSTEM
1. Add Employee
2. Update Employee
3. Delete Employee
4. Search Employee
5. List All Employees
6. Exit
